## Initialization

In [268]:

import yfinance as yf
import pandas as pd
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np
from plotly.subplots import make_subplots
import os
from jinja2 import Template
import json
import markdown2
from IPython.display import HTML




In [269]:
## Choose the date you want to generate Analysis
## You should have csv including: 
## 1. The daily performance: data/Performance/Performance_{date}.csv,    
## 2. The daily trading data : data/TradingData/nasdaq_100_5min_data_{date}.csv, 
## 3. The previous day trading data : data/TradingData/nasdaq_100_5min_data_{date_previous_str}.csv
date = '2024-04-12'

In [270]:
## considering the previous date could be last week friday, but if there is holiday, you should adjust this part manually
# Convert the date string to a datetime object
date_obj = datetime.strptime(date, '%Y-%m-%d')

# Check if the given date is a Monday
if date_obj.weekday() == 0:  # Monday is represented by 0
    # If it's Monday, subtract 3 days to get the previous working day
    date_previous = date_obj - timedelta(days=3)
else:
    # If it's not Monday, subtract 1 day to get the previous working day
    date_previous = date_obj - timedelta(days=1)

# Convert the datetime object back to a string in the same format
date_previous_str = date_previous.strftime('%Y-%m-%d')

In [271]:
## load data
trade = pd.read_csv(f'data/Performance/Performance_{date}.csv')
df = pd.read_csv(f'data/TradingData/nasdaq_100_5min_data_{date}.csv')
try:
    df_previous = pd.read_csv(f'data/TradingData/nasdaq_100_5min_data_{date_previous_str}.csv')
except FileNotFoundError as e:
    df_previous = df.copy()
    print(f"Cannot find file '{e.filename}'")


In [272]:
## Preprocess trade to trade_rth

trade = trade.drop(columns=['symbol','_priceFormat', '_priceFormatType', '_tickSize', 'buyFillId','sellFillId'])
# 1. Convert pnl to numeric format with proper handling of negative values
trade['pnl'] = trade['pnl'].str.replace('$', '')
trade['pnl'] = trade['pnl'].str.replace('(', '-').str.replace(')', '').astype(float)

# 2. Convert boughtTimestamp and soldTimestamp to datetime format and adjust to UTC
trade['boughtTimestamp'] = pd.to_datetime(trade['boughtTimestamp']) - pd.Timedelta(hours=7)
trade['soldTimestamp'] = pd.to_datetime(trade['soldTimestamp']) - pd.Timedelta(hours=7)

# 3. Convert duration to time format
trade['duration'] = pd.to_timedelta(trade['duration'])

#4. Filter out trades with boughtTimestamp or soldTimestamp between 9:30 and 16:10
trade_rth = trade[(trade['boughtTimestamp'].dt.time >= pd.Timestamp('09:30').time()) &
                       (trade['boughtTimestamp'].dt.time <= pd.Timestamp('16:10').time()) |
                       (trade['soldTimestamp'].dt.time >= pd.Timestamp('09:30').time()) &
                       (trade['soldTimestamp'].dt.time <= pd.Timestamp('16:10').time())].copy()  # Ensure a copy of the DataFrame is created

trade_rth['boughtTimestamp'] = trade_rth['boughtTimestamp'].dt.floor('min')
trade_rth['soldTimestamp'] = trade_rth['soldTimestamp'].dt.floor('min')

trade_rth['boughtTimestamp'] = trade_rth['boughtTimestamp'].dt.floor('5min')
trade_rth['soldTimestamp'] = trade_rth['soldTimestamp'].dt.floor('5min')


In [273]:
## Preprocess TradingData(df) to df_rth

# 0. Convert the 'Datetime' column to datetime dtype
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)

# 1. Convert the 'Datetime' column to datetime if it's not already in datetime format
df.index = pd.to_datetime(df.index)

# 2. Filter rows within the specified time range directly using the index
df_rth = df[(df.index.time >= pd.Timestamp('09:30').time()) & (df.index.time <= pd.Timestamp('16:10').time())]

In [274]:
## Preprocess previous TradingData(df_previous) to df_previous_rth

# 0. Convert the 'Datetime' column to datetime dtype
df_previous['Datetime'] = pd.to_datetime(df_previous['Datetime'])
df_previous.set_index('Datetime', inplace=True)

# 1. Convert the 'Datetime' column to datetime if it's not already in datetime format
df_previous.index = pd.to_datetime(df_previous.index)

# 2. Filter rows within the specified time range directly using the index
df_previous_rth = df_previous[(df_previous.index.time >= pd.Timestamp('09:30').time()) & (df_previous.index.time <= pd.Timestamp('16:10').time())]


# 3. get previous open, close, highest and lowest
# Find the overall highest and lowest values for High and Low
pre_high = df_previous_rth['High'].max()
pre_low = df_previous_rth['Low'].min()

pre_open = df_previous_rth['Open'].iloc[0]
pre_close = df_previous_rth['Close'].iloc[-1]

pre_market = {
    'pre_high': pre_high,
    'pre_low': pre_low,
    'pre_open': pre_open,
    'pre_close': pre_close
}

# 4. Get trhe last 20 bars
df_previous_rth = df_previous_rth[-20:]

In [275]:
## Calculate rth EMA including last 20 of df_previous_rth

# Combine previous and current trading session data
combined_df = pd.concat([df_previous_rth, df_rth])

# Calculate the 20-period EMA on the combined close prices
combined_df['EMA_20'] = combined_df['Close'].ewm(span=20, adjust=False).mean().round(2)

# # Now, slice back out the EMA values that correspond only to df_rth dates
df_rth['EMA_20'] = combined_df['EMA_20'].iloc[-len(df_rth):].values



/var/folders/cp/1g7t_8rd6kq_6f843vp6sg700000gn/T/ipykernel_2423/3164182010.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [276]:
# Calculate winning and losing trades
winning_trades = trade_rth[trade_rth['pnl'] > 0]
losing_trades = trade_rth[trade_rth['pnl'] <= 0]

# Data for pie chart
win_loss_counts = {'Winning Trades': len(winning_trades), 'Losing Trades': len(losing_trades)}




In [277]:
# Generate statistics for tables
def trade_stats(trades):
    if len(trades) == 0:
        return {'Average PnL': 0, 'Max PnL': 0, 'Min PnL': 0, 'Total PnL': 0, 'Count': 0}
    return {
        'Average PnL': round(trades['pnl'].mean(), 2),
        'Max PnL': round(trades['pnl'].max(), 2),
        'Min PnL': round(trades['pnl'].min(), 2),
        'Total PnL': round(trades['pnl'].sum(), 2),
        'Count': len(trades)
    }

all_trades_stats = trade_stats(trade_rth)
winning_trades_stats = trade_stats(winning_trades)
losing_trades_stats = trade_stats(losing_trades)

In [278]:
## Until now, you should have the following data for plotting

# all_trades_stats 
# winning_trades_stats 
# losing_trades_stats
# df_rth
# trade_rth
# pre_high
# pre_low 
# pre_open 
# pre_close 
# win_loss_counts

## Preparing Plots

In [279]:
def create_candlestick_trace(df):
    return go.Candlestick(
        x=df.index.strftime('%Y-%m-%d %H:%M'),
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close']
    )

def create_pie_trace(win_loss_counts):
    labels = list(win_loss_counts.keys())
    values = list(win_loss_counts.values())
    return go.Pie(labels=labels, values=values, showlegend=False ,name="Win vs Loss")

def create_trade_lines(trade_df):
    trade_lines = []
    for index, row in trade_df.iterrows():
        entry_time, exit_time = sorted([row['boughtTimestamp'], row['soldTimestamp']])
        entry_price, exit_price = (row['buyPrice'], row['sellPrice']) if entry_time == row['boughtTimestamp'] else (row['sellPrice'], row['buyPrice'])
        color = 'green' if row['pnl'] > 0 else 'red'
        trade_line = go.Scatter(
            x=[entry_time, exit_time],
            y=[entry_price, exit_price],
            mode='lines+markers',
            line=dict(color=color, width=2),
            marker=dict(color=color, size=8),
            name=f"Trade {index}",
            hoverinfo='text',
            hovertext=f"Entry: {entry_time}, Price: {entry_price} | Exit: {exit_time}, Price: {exit_price}, PnL: {row['pnl']}"
        )
        trade_lines.append(trade_line)
    return trade_lines

def create_ema_trace(df):
    return go.Scatter(
        x=df.index.strftime('%Y-%m-%d %H:%M'),
        y=df['EMA_20'],
        mode='lines',
        line=dict(color='blue', width=2),
        name='EMA 20'
    )


In [280]:
def create_horizontal_lines(df, values, colors, labels):
    return [
        go.Scatter(
            x=[df.index.min(), df.index.max()],
            y=[value] * 2,
            mode='lines',
            line=dict(color=color, width=2, dash='dash'),
            name=f'pre_{label.lower()}'
        ) for value, color, label in zip(values, colors, labels)
    ]

def create_annotations(df):
    low_points = df['Low']
    offset = low_points.min() * 0.001
    return [
        {'x': date, 'y': low - offset, 'xref': 'x', 'yref': 'y', 'text': str(i),
         'showarrow': False, 'font': {'family': 'Arial, sans-serif', 'size': 12, 'color': 'black'}, 'align': 'center'}
        for i, (date, low) in enumerate(zip(df.index, low_points), start=1)
    ]


In [281]:
def assemble_plot(df, trade_df, pre_market, pre_colors):
    fig = go.Figure()
    fig.add_trace(create_candlestick_trace(df))
    
    fig.add_trace(create_ema_trace(df))
    fig.add_traces(create_horizontal_lines(df, list(pre_market.values()), pre_colors, list(pre_market.keys())))
    fig.add_traces(create_trade_lines(trade_df))
    fig.update_layout(
        title=f'NASDAQ 100 Futures Candlestick Chart - {df.index.date[0]}',
        xaxis_title='Datetime', yaxis_title='Price',
        xaxis_rangeslider_visible=False, xaxis=dict(tickangle=-45),
        yaxis=dict(tickformat='none'), annotations=create_annotations(df)
    )
    return fig

def create_pie_chart(win_loss_counts, all_trades_stats, winning_trades_stats, losing_trades_stats):
    # Define the layout with 1 column for the pie and 1 column for the tables, split into 3 rows
    fig = make_subplots(
        rows=3, cols=2,
        specs=[[{"type": "pie", "rowspan": 3}, {"type": "table"}],
               [None, {"type": "table"}],
               [None, {"type": "table"}]],
        column_widths=[0.4, 0.6],
        subplot_titles=(None, "All Trades", "Winning Trades", "Losing Trades")
    )

    # Add the pie chart in the first column, spanning all three rows
    pie_trace = create_pie_trace(win_loss_counts)
    fig.add_trace(pie_trace, row=1, col=1)

    # Helper function to create table traces
    def create_table_trace(data, header_color='lightgrey'):
        headers = list(data.keys())
        values = [[v] for v in data.values()]
        return go.Table(
            header=dict(values=headers, fill_color=header_color, align='left'),
            cells=dict(values=values, align='left')
        )

    # Add table for all trades statistics
    all_trades_table = create_table_trace(all_trades_stats)
    fig.add_trace(all_trades_table, row=1, col=2)

    # Add table for winning trades statistics
    winning_trades_table = create_table_trace(winning_trades_stats, header_color='lightgreen')
    fig.add_trace(winning_trades_table, row=2, col=2)

    # Add table for losing trades statistics
    losing_trades_table = create_table_trace(losing_trades_stats, header_color='salmon')
    fig.add_trace(losing_trades_table, row=3, col=2)

    # Update layout to fit the table sizes and remove empty subplot titles
    fig.update_layout(
        title='Trade Outcome Distribution and Statistics',
        showlegend=True
    )

    return fig




def create_summary(summary_file, date):
    # Convert the provided date string to a datetime object for easier comparison
    target_date = datetime.strptime(date, '%Y-%m-%d').date()
    section_found = False
    content = []
    
    with open(summary_file, 'r', encoding='utf-8') as file:
        for line in file:
            # Check for section headers
            if line.startswith('## '):
                # Extract the date from the section header
                section_date = datetime.strptime(line.strip()[3:], '%Y-%m-%d').date()
                if section_date == target_date:
                    section_found = True
                    continue
                elif section_found:
                    # If another section starts, stop reading
                    break
            
            if section_found:
                # Collect all lines after the target date section until another section starts
                content.append(line)
    html_content = markdown2.markdown(''.join(content))

    return html_content




In [282]:
def generate_html(fig_assemble, fig_statistic, html_summary, folder_path):
    toggle_buttons = """
    <div style='margin-top: 20px;'>
        <button onclick='toggleAllTrades()'>Trading Details</button>
        <button onclick='toggleSummary()'>Summary</button>
    </div>
    """

    toggle_script = """
    <script>
    function toggleAllTrades() {
        var plotElement = document.getElementById('plot_assemble');
        var statisticElement = document.getElementById('plot_statistic');
        var tradingTracesStartIndex = 6; // Assuming that trading traces start at index 1
        var tradingTracesEndIndex = plotElement.data.length - 1; // Assuming last trace is not a trading trace
        for (var i = tradingTracesStartIndex; i <= tradingTracesEndIndex; i++) {
            var currentVisibility = plotElement.data[i].visible;
            var newVisibility = (currentVisibility === 'legendonly' ? true : 'legendonly');
            Plotly.restyle(plotElement, {'visible': newVisibility}, [i]);
        }
        // Toggle entire plot_statistic visibility
        statisticElement.style.display = (statisticElement.style.display === 'none' ? 'block' : 'none');
    }
    function toggleSummary() {
        var summaryElement = document.getElementById('plot_summary');
        summaryElement.style.display = (summaryElement.style.display === 'none' ? 'block' : 'none');
    }
    </script>
    """
    
    css_styles = """
    <style>
        #plot_assemble, #plot_statistic, #plot_summary {
            width: 95vw; 
            height: 95vh; 
            transition: width 0.5s ease-in-out, height 0.5s ease-in-out; /* Smooth transition for resizing */
        }
    </style>
    """

    resize_script = """
    <script>
    function resizePlot() {
            var width = window.innerWidth * 0.95;
            var height = window.innerHeight * 0.95;
            var plotElements = [document.getElementById('plot_assemble'), document.getElementById('plot_statistic'), document.getElementById('plot_summary')];

            plotElements.forEach(function(elem) {
                elem.style.width = width + 'px';
                elem.style.height = height + 'px';
                Plotly.relayout(elem, {
                    width: width,
                    height: height
                });
            });
        }

        window.onresize = resizePlot;
        window.onload = resizePlot;
    </script>
    """

    html_content = f"""
    <html>
    <head>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        {css_styles}
    </head>
    <body>
        <div id='plot_assemble'></div>
        {resize_script}
        {toggle_buttons}
        <div id='plot_statistic' style='margin-top: 20px;'></div>
        <div id='plot_summary' style='margin-top: 20px; display: none;'>{html_summary}</div> <!-- Initially hidden -->
        <script>
            var plotElementAssemble = document.getElementById('plot_assemble');
            var dataAssemble = {fig_assemble.to_json()};
            Plotly.newPlot(plotElementAssemble, dataAssemble.data, dataAssemble.layout, {{scrollZoom: true}});
            var plotElementStatistic = document.getElementById('plot_statistic');
            var dataStatistic = {fig_statistic.to_json()};
            Plotly.newPlot(plotElementStatistic, dataStatistic.data, dataStatistic.layout);
            plotElementAssemble.on('plotly_afterplot', function() {{
            // Target the y-axis area for adding a wheel event listener
                var yAxisArea = plotElementAssemble.querySelector('.yaxislayer-above');
                if (yAxisArea) {{
                    yAxisArea.addEventListener('wheel', function(event) {{
                        event.preventDefault(); // Prevent the default scrolling action

                        var deltaY = event.deltaY * -0.005; // Adjust this factor to control zoom speed
                        var yaxis = plotElementAssemble.layout.yaxis;

                        // Calculate the new range based on the scroll direction and zoom factor
                        var rangeDiff = (yaxis.range[1] - yaxis.range[0]) * deltaY;
                        var newRange = [yaxis.range[0] - rangeDiff, yaxis.range[1] + rangeDiff];

                        // Update the y-axis range without affecting the x-axis
                        Plotly.relayout(plotElementAssemble, {{'yaxis.range': newRange}});
                    }});
                }}
            }});
        </script>

        {toggle_script}

    </body>
    </html>
    """

    file_name = f"{folder_path}NASDAQ_100_Futures_Candlestick_Chart_{df.index.date[0]}.html"
    with open(file_name, "w") as f:
        f.write(html_content)

# Usage
folder_path = "./data/HTML/"
fig_assemble = assemble_plot(df_rth, trade_rth, pre_market, ['orange', 'purple', 'green', 'red'])
fig_statistic = create_pie_chart(win_loss_counts, all_trades_stats, winning_trades_stats, losing_trades_stats)
html_summary= create_summary('./data/HTML/Summary.md', date)
generate_html(fig_assemble, fig_statistic, html_summary, folder_path)


## overall statistical data

In [283]:
# Path to the directory containing CSV files
directory = './data/Performance'

# Initialize an empty list to store DataFrames
dfs = []

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        filepath = os.path.join(directory, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(filepath)
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames vertically
df_overall = pd.concat(dfs, ignore_index=True)


In [284]:
## Preprocess trade to trade_rth

df_overall = df_overall.drop(columns=['symbol','_priceFormat', '_priceFormatType', '_tickSize', 'buyFillId','sellFillId'])
# 1. Convert pnl to numeric format with proper handling of negative values
df_overall['pnl'] = df_overall['pnl'].str.replace('$', '')
df_overall['pnl'] = df_overall['pnl'].str.replace('(', '-').str.replace(')', '').astype(float)

# 2. Convert boughtTimestamp and soldTimestamp to datetime format and adjust to UTC
df_overall['boughtTimestamp'] = pd.to_datetime(df_overall['boughtTimestamp']) - pd.Timedelta(hours=7)
df_overall['soldTimestamp'] = pd.to_datetime(df_overall['soldTimestamp']) - pd.Timedelta(hours=7)

# 3. Convert duration to time format
df_overall['duration'] = pd.to_timedelta(df_overall['duration'])


In [285]:
# Calculate winning and losing trades
overall_winning_trades = df_overall[df_overall['pnl'] > 0]
overall_losing_trades = df_overall[df_overall['pnl'] <= 0]

# Data for pie chart
overall_win_loss_counts = {'Winning Trades': len(overall_winning_trades), 'Losing Trades': len(overall_losing_trades)}

In [286]:
overall_trades_stats = trade_stats(df_overall)
overall_winning_trades_stats = trade_stats(overall_winning_trades)
overall_losing_trades_stats = trade_stats(overall_losing_trades)

In [287]:
def overall_generate_html(fig_statistic, folder_path):
    toggle_buttons = """
    <div style='margin-top: 20px;'>
        <button onclick='toggleAllTrades()'>Trading Details</button>
    </div>
    """

    toggle_script = """
    <script>
    function toggleAllTrades() {
        var statisticElement = document.getElementById('plot_statistic');
        // Toggle entire plot_statistic visibility
        statisticElement.style.display = (statisticElement.style.display === 'none' ? 'block' : 'none');
    }
    </script>
    """

    resize_script = """
    <script>
    function resizePlot() {

        var plotElement_statistic = document.getElementById('plot_statistic');

        plotElement_statistic.style.width = window.innerWidth * 0.9 + 'px';
        plotElement_statistic.style.height = window.innerHeight * 0.9 + 'px';

        Plotly.relayout(plotElement_statistic, {
            width: window.innerWidth * 0.9,
            height: window.innerHeight * 0.9
        });
    }
    window.onresize = resizePlot;
    window.onload = resizePlot;
    </script>
    """

    html_content = f"""
    <html>
    <head><script src="https://cdn.plot.ly/plotly-latest.min.js"></script></head>
    <body>

        {resize_script}
        {toggle_buttons}
        <div id='plot_statistic' style='width: 100%; height: 100%;'></div>
        <script>
            var plotElementStatistic = document.getElementById('plot_statistic');
            var dataStatistic = {fig_statistic.to_json()};
            Plotly.newPlot(plotElementStatistic, dataStatistic.data, dataStatistic.layout);
        </script>
        {toggle_script}

    </body>
    </html>
    """

    file_name = f"{folder_path}Overall_Performance.html"
    with open(file_name, "w") as f:
        f.write(html_content)

# Usage
folder_path = "./data/HTML/"
fig_statistic = create_pie_chart(overall_win_loss_counts, overall_trades_stats, overall_winning_trades_stats, overall_losing_trades_stats)
overall_generate_html( fig_statistic, folder_path)


## Generate Index

In [288]:
# Directory where the HTML files are stored
directory = "./data/HTML/"
# Root directory for the index.html
root_directory = "./"

# List HTML files excluding index.html
files = sorted([f for f in os.listdir(directory) if f.endswith('.html') and f != 'index.html'])

# Generate HTML for the index file
index_html = """
<html>
<head>
    <title>NASDAQ 100 Futures Charts Index</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            display: flex;
            height: 100vh;
            margin: 0; /* Ensure no margin */
            padding: 0; /* Ensure no padding */
        }

        #navbar {
            overflow-y: auto;
            width: 150px; /* Fixed width of the navigation bar */
            height: 100%;
            border-right: 1px solid #ccc;
        }

        #content {
            flex-grow: 1; /* Content area takes up remaining space */
        }

        iframe {
            width: 100%;
            height: 100%;
            border: none;
        }


    </style>
</head>
<body>
    <div id="navbar">
        <ul style="list-style-type: none; padding: 20px;">
"""

# Creating links that point to the HTML files in the data/HTML directory
for file in files:
    display_name = file.split('_')[-1].replace('.html', '')
    index_html += f'<li><a href="{file}" target="contentFrame">{display_name}</a></li>\n'

index_html += """
        </ul>
    </div>
    <div id="content">
        <iframe name="contentFrame"></iframe>
    </div>
</body>
</html>
"""

# Save the index.html in the root directory
index_file_path = os.path.join(directory, "index.html")
with open(index_file_path, "w") as f:
    f.write(index_html)
